In [1]:
# loading data

import torch
import copy

file_path = 'data/cora_fixed_sbert.pt'
data_org = torch.load(file_path, map_location='cpu')

# org data for test
#print(data_org.y[14])

# data for train
data_train = copy.deepcopy(data_org)  
print(data_train.label_names)



/tmp/ipykernel_782486/765789076.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data_org = torch.load(file_path, map_location='cpu')


['Rule_Learning', 'Neural_Networks', 'Case_Based', 'Genetic_Algorithms', 'Theory', 'Reinforcement_Learning', 'Probabilistic_Methods']


/home/chunjielu/miniconda3/envs/faiss_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
file_path = 'LLM_GNN_data/cora^cache^consistency.pt'
LLM_label = torch.load(file_path, map_location='cpu')
op=0
total=0
for i in range(2708):
    if LLM_label['conf'][i].item()<0.90:
        continue
    total+=1
    if LLM_label['pred'][i].item()==data_org.y[i].item():
        op+=1
        #print(i,LLM_label['conf'][i].item())
#print(op/total , total)
print(f'accuracy {op/total}, number {total}')

accuracy 0.8918918918918919, number 37


/tmp/ipykernel_782486/2059984175.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  LLM_label = torch.load(file_path, map_location='cpu')


In [4]:
# set the test_data for test

def random_split_masks(data, num_train=140, num_val=500, num_test=1000, seed=42):
    torch.manual_seed(seed)  

    num_nodes = data.num_nodes
    all_indices = torch.randperm(num_nodes) 


    train_idx = all_indices[:num_train]
    val_idx = all_indices[num_train:num_train + num_val]
    test_idx = all_indices[num_train + num_val:num_train + num_val + num_test]

    data.train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    data.val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    data.test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    data.train_mask[train_idx] = True
    data.val_mask[val_idx] = True
    data.test_mask[test_idx] = True

    return data 
data_org = random_split_masks(data_org)
data_train = random_split_masks(data_train)

print("Train size:", data_train.train_mask.sum().item())
print("Val size:", data_train.val_mask.sum().item())
print("Test size:", data_train.test_mask.sum().item())
print("是否有交集:", (data_train.train_mask & data_train.val_mask).any().item() or 
                   (data_train.train_mask & data_train.test_mask).any().item() or 
                   (data_train.val_mask & data_train.test_mask).any().item())

Train size: 140
Val size: 500
Test size: 1000
是否有交集: False


In [6]:
# difficulty aware selection

import networkx as nx
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx
import numpy as np
from sklearn.cluster import KMeans
from torch_geometric.datasets import Planetoid
from sklearn.metrics.pairwise import euclidean_distances


def get_rank_score(rank_list,n):
    result=[0 for i in range(n)]
    for i in range(n):
        k=rank_list[i]
        result[k]=100*(n-i)/n
    return result

def compute_pagerank_and_ranking(data):
    """
    page rank ranking
    """
    G = to_networkx(data, to_undirected=True)  


    pagerank = nx.pagerank(G, alpha=0.85)

 
    sorted_nodes = sorted(pagerank.items(), key=lambda x: x[1], reverse=True)

  
    rank_list = [node for node, rank in sorted_nodes]

    return get_rank_score(rank_list,data.num_nodes)
import faiss
from torch_geometric.utils import to_undirected


def compute_rC_density(node_features: torch.Tensor, cluster_centers: torch.Tensor, labels: torch.Tensor):

    assigned_centers = cluster_centers[labels]       
    distances = torch.norm(node_features - assigned_centers, dim=1)  
    c_density = 1 / (1 + distances)                   
    return c_density


def compute_cluster_rank(data, num_clusters=7):


    x = data.x.detach().cpu().float()
    n, d = x.shape

  
    kmeans = faiss.Kmeans(d=d, k=num_clusters, niter=20, verbose=False, seed=42)
    kmeans.train(x.numpy())

    
    _, I = kmeans.index.search(x.numpy(), 1)  
    labels = torch.tensor(I.squeeze(), dtype=torch.long) 

    centers = torch.tensor(kmeans.centroids, dtype=x.dtype) 

    c_density = compute_rC_density(x, centers, labels) 

    sorted_idx = torch.argsort(c_density, descending=True)  

    rank_list = sorted_idx.tolist()
    return get_rank_score(rank_list, data.num_nodes)



n=data_train.num_nodes
#print(n)
page_list=compute_pagerank_and_ranking(data_train)
cluster_list=compute_cluster_rank(data_train)


result_score=[]
for i in range(n):
    score=page_list[i]*0.5+cluster_list[i]*0.5
    result_score.append((i,score))

#print(result_score)

sorted_result=sorted(result_score, key=lambda x: x[1], reverse=True)
rank_list = [node for node, _ in sorted_result]
#rank_list = rank_list[::-1]
print(rank_list)


[2178, 1686, 2583, 753, 31, 1982, 1880, 1896, 2036, 1548, 1516, 2406, 2159, 1864, 1634, 2065, 2167, 1554, 927, 2467, 962, 2175, 1163, 1214, 2614, 949, 2117, 312, 1164, 1633, 1595, 1642, 893, 1627, 990, 1390, 1507, 1981, 2174, 2215, 1118, 1462, 2216, 2559, 1528, 1983, 2439, 1070, 1637, 1269, 1536, 2264, 1053, 1857, 56, 1260, 2177, 1372, 58, 1635, 1369, 964, 1047, 1171, 1948, 1020, 921, 2571, 2612, 961, 2058, 16, 1499, 1610, 1701, 1620, 1569, 449, 2014, 2010, 2513, 68, 1052, 11, 2654, 997, 1584, 1914, 2376, 2481, 1416, 1762, 1953, 1840, 2688, 2340, 940, 1588, 2300, 2099, 2173, 69, 2530, 1711, 2105, 1440, 2183, 2650, 1998, 2186, 1892, 2628, 1761, 1270, 1578, 1207, 1524, 178, 1671, 1741, 699, 1134, 1263, 2348, 1413, 977, 1927, 1932, 2343, 1011, 1926, 2181, 889, 2381, 2082, 1005, 1592, 1665, 1791, 2123, 1717, 2492, 1437, 1626, 1763, 2627, 2635, 1593, 1267, 933, 1344, 1764, 2304, 1153, 1629, 2043, 1147, 1604, 2164, 2253, 2613, 2333, 2278, 1787, 1863, 0, 2676, 1558, 446, 1101, 1653, 1697, 174

In [ ]:
"""
filter_list=[]
total=0
op=0
for i in rank_list:
    if LLM_label['conf'][i].item()>=0.88:
        total+=1
        filter_list.append(i)
        if LLM_label['pred'][i].item()==data_org.y[i].item():
            op+=1
print(len(filter_list))
print(op/total)
"""


608
0.8898026315789473
